In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
from catboost import CatBoostClassifier

# 데이터 로드
data = pd.read_csv("/Users/whovi/Downloads/monunmon.csv")

# Label 값에 1을 더하여 0부터 시작하는 연속된 정수로 변환
data['Label'] = data['Label'] + 1

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# CatBoost는 DataFrame 형태를 권장하므로 다시 DataFrame으로 변환
X_train = pd.DataFrame(X_train, columns=X.columns)
X_val = pd.DataFrame(X_val, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

### 기본 CatBoost 모델 ###
#print("=== Default CatBoost ===")
#model_default = CatBoostClassifier(random_seed=42, eval_metric='MultiClass', loss_function='MultiClass', verbose=0)
#model_default.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, early_stopping_rounds=50)

# 테스트 데이터 평가
#y_test_pred_default = model_default.predict(X_test)

# 성능 출력
#print("\nClassification Report on Test Data (Default CatBoost):")
#print(classification_report(y_test, y_test_pred_default))

#test_accuracy_default = accuracy_score(y_test, y_test_pred_default)
#test_f1_default = f1_score(y_test, y_test_pred_default, average='weighted')

#print(f"Test Accuracy: {test_accuracy_default}")
#print(f"Test F1 Score: {test_f1_default}")

### GridSearchCV를 이용한 하이퍼파라미터 튜닝 ###
param_grid = {
    'depth': [4, 6, 8],               # 트리 깊이
    'learning_rate': [0.1, 0.2], # 학습률
    'iterations': [500, 1000],         # 반복 횟수
    'l2_leaf_reg': [1, 3],          # L2 정규화
}

grid_model = CatBoostClassifier(
    random_seed=42,
    loss_function='MultiClass',
    eval_metric='MultiClass',
    verbose=0
)

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=grid_model,
    param_grid=param_grid,
    scoring='accuracy',  # 평가 기준
    cv=3,                # 3-겹 교차 검증
    verbose=1,
    n_jobs=-1            # 병렬 처리
)

# GridSearchCV 실행
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

# 최적 모델로 테스트 데이터 평가
best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)

# 최적 모델 성능 출력
print("\nClassification Report on Test Data (Tuned CatBoost):")
print(classification_report(y_test, y_test_pred))

test_accuracy_tuned = accuracy_score(y_test, y_test_pred)
test_f1_tuned = f1_score(y_test, y_test_pred, average='weighted')

print(f"Test Accuracy (Tuned): {test_accuracy_tuned}")
print(f"Test F1 Score (Tuned): {test_f1_tuned}")


Fitting 3 folds for each of 24 candidates, totalling 72 fits
